<a href="https://colab.research.google.com/github/rayeeed/UFP_Project/blob/main/PM2_5_Prediction_datagen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import IsolationForest
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from xgboost import XGBRegressor
#from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import cross_val_score,cross_val_predict
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import RidgeCV
from sklearn.feature_selection import mutual_info_regression
from sklearn.feature_selection import SelectFromModel
from time import time
from sklearn.linear_model import RidgeCV
from sklearn.model_selection import RepeatedKFold
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error
import seaborn as sns
from sklearn.preprocessing import Normalizer,MinMaxScaler,StandardScaler,RobustScaler
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Ridge, Lasso, BayesianRidge
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.feature_selection import RFE,RFECV
from sklearn.linear_model import (HuberRegressor,
                              	RANSACRegressor, TheilSenRegressor)

In [ ]:
!pip install optuna
import optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.1/380.1 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.0/233.0 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 1.8 MB/s eta 0:00:00


In [ ]:
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.preprocessing import KBinsDiscretizer

# Load the dataset
df = pd.read_excel('/content/drive/MyDrive/datasets_Rayeed_GSV_POI_LU/model_35.xlsx', index_col='site_name')

# Drop unnecessary columns
df.drop(['Site_type', 'OID'], inplace=True, axis=1)

# Separate the target variable and features
y = df['Mean PM2.5 (µg/m3)']
X = df.drop(['Mean PNC (# / cm3)', 'Mean PM2.5 (µg/m3)'], axis=1)

In [ ]:
# Discretize the target variable into bins using 'quantile' strategy
binner = KBinsDiscretizer(n_bins=5, encode='ordinal', strategy='quantile')
y_binned = binner.fit_transform(y.values.reshape(-1, 1)).flatten()

# Perform stratified train-test split
split = StratifiedShuffleSplit(n_splits=1, test_size=.2, random_state=42)
for train_index, test_index in split.split(X, y_binned):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

# Check the distribution of bins in the train and test sets
print("Train bin distribution:\n", pd.value_counts(y_binned[train_index], sort=False))
print("Test bin distribution:\n", pd.value_counts(y_binned[test_index], sort=False))

# Output the resulting dataframes
print("X_train:\n", X_train.head())
print("y_train:\n", y_train.head())
print("X_test:\n", X_test.head())
print("y_test:\n", y_test.head())

Train bin distribution:
 0.0    6
3.0    7
4.0    5
2.0    5
1.0    5
Name: count, dtype: int64
Test bin distribution:
 1.0    1
0.0    1
2.0    1
4.0    2
3.0    2
Name: count, dtype: int64
X_train:
                      gsv_wall_p100  gsv_building_p100  gsv_house_p100  \
site_name                                                               
Dhanmondi_lake_park         0.3000             1.3425          0.0025   
Chandrima_park              1.9925             0.2625          0.0000   
Aftabnagar                  3.6900            32.4275          0.2575   
Technical                   2.4050             8.3575          0.3650   
Tejgaon                     4.0325            16.0975          0.0000   

                     gsv_awning_p100  gsv_sky_p100  gsv_earth_p100  \
site_name                                                            
Dhanmondi_lake_park           0.0000       17.1325         20.7575   
Chandrima_park                0.0000       40.2650          1.3275   
Aftabna

<ipython-input-4-a1486702b7a8>:12: FutureWarning: pandas.value_counts is deprecated and will be removed in a future version. Use pd.Series(obj).value_counts() instead.
  print("Train bin distribution:\n", pd.value_counts(y_binned[train_index], sort=False))
<ipython-input-4-a1486702b7a8>:13: FutureWarning: pandas.value_counts is deprecated and will be removed in a future version. Use pd.Series(obj).value_counts() instead.
  print("Test bin distribution:\n", pd.value_counts(y_binned[test_index], sort=False))


In [ ]:
from scipy.stats import ks_2samp
ks_statistic, p_value = ks_2samp(y_train, y_test)

print(f"K-S statistic: {ks_statistic}")
print(f"P-value: {p_value}")

# Interpretation
alpha = 0.05
if p_value < alpha:
    print("The null hypothesis is rejected. The distributions of the train and test sets are different.")
else:
    print("The null hypothesis cannot be rejected. The distributions of the train and test sets are the same.")

K-S statistic: 0.32142857142857145
P-value: 0.5556032252116138
The null hypothesis cannot be rejected. The distributions of the train and test sets are the same.


In [ ]:
# Apply Kolmogorov-Smirnov test for each feature
ks_results = {}
failed_features = []
alpha = 0.05
for feature in X.columns:
    ks_statistic, p_value = ks_2samp(X_train[feature], X_test[feature])
    ks_results[feature] = {'ks_statistic': ks_statistic, 'p_value': p_value}
    if p_value < alpha:
        failed_features.append(feature)

# Print K-S test results for each feature
for feature, result in ks_results.items():
    print(f"Feature: {feature}")
    print(f"K-S statistic: {result['ks_statistic']}")
    print(f"P-value: {result['p_value']}")
    if result['p_value'] < alpha:
        print("The null hypothesis is rejected. The distributions of the train and test sets are different for this feature.\n")
    else:
        print("The null hypothesis cannot be rejected. The distributions of the train and test sets are the same for this feature.\n")

# Print features that failed the K-S test
if failed_features:
    print("Features that failed the K-S test (null hypothesis rejected):")
    for feature in failed_features:
        print(f"- {feature}")
else:
    print("All features passed the K-S test (null hypothesis not rejected).")

Feature: gsv_wall_p100
K-S statistic: 0.32142857142857145
P-value: 0.5556032252116138
The null hypothesis cannot be rejected. The distributions of the train and test sets are the same for this feature.

Feature: gsv_building_p100
K-S statistic: 0.4642857142857143
P-value: 0.14808640616728036
The null hypothesis cannot be rejected. The distributions of the train and test sets are the same for this feature.

Feature: gsv_house_p100
K-S statistic: 0.2857142857142857
P-value: 0.7050303962215889
The null hypothesis cannot be rejected. The distributions of the train and test sets are the same for this feature.

Feature: gsv_awning_p100
K-S statistic: 0.17857142857142858
P-value: 0.9883820703931286
The null hypothesis cannot be rejected. The distributions of the train and test sets are the same for this feature.

Feature: gsv_sky_p100
K-S statistic: 0.35714285714285715
P-value: 0.4252563751762208
The null hypothesis cannot be rejected. The distributions of the train and test sets are the same

In [ ]:
X_train_dropped = X_train.drop(columns=failed_features)
X_test_dropped = X_test.drop(columns=failed_features)

In [ ]:
trans = StandardScaler()
X_st = trans.fit_transform(X_train_dropped)
X_st = pd.DataFrame(X_st, columns=X_train_dropped.columns, index=X_train_dropped.index)

In [ ]:
X_test_st = trans.transform(X_test_dropped)
X_test_st = pd.DataFrame(X_test_st, columns=X_test_dropped.columns, index=X_test_dropped.index)

In [ ]:
X_Pred = pd.read_excel('/content/drive/MyDrive/datasets_Rayeed_GSV_POI_LU/combined_total_v4.0.xlsx', index_col='OID_')

In [ ]:
col=X_st.columns
X_Pred_f1 = X_Pred[col]
X_Pred_f2= X_Pred_f1

In [ ]:
x_pred_st= trans.transform(X_Pred_f2)
x_pred_st = pd.DataFrame(x_pred_st, columns = X_Pred_f2.columns,index=X_Pred_f2.index)

# **SVR**

In [ ]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import cross_val_predict


# Feature set to be used for prediction
features = ['gsv_tree_p750', 'RdclP_250', 'build_250', 'gsv_sidewalk_p250']

# Function to run the optimization and evaluate the model
def evaluate_model(features):
    X_Train = X_st[features]
    X_test = X_test_st[features]

    def objective(trial):
        C = trial.suggest_float("C", 0.01, 1000000.0, log=True)
        epsilon = trial.suggest_float("epsilon", 0.001, 10.0)
        gamma = trial.suggest_float("gamma", 0.0001, 100000, log=True)

        svr = SVR(C=C, epsilon=epsilon, gamma=gamma)
        predicted = cross_val_predict(svr, X_Train, y_train, cv=10)
        return mean_squared_error(y_train, predicted, squared=False)

    study = optuna.create_study(direction='minimize', study_name="SVR_RMSE")
    study.optimize(objective, n_trials=500)  # Run for 500 iterations

    best_params = study.best_params
    model = SVR(C=best_params["C"], epsilon=best_params["epsilon"], gamma=best_params["gamma"])
    model.fit(X_Train, y_train)

    pred_train = model.predict(X_Train)
    rmse_train = mean_squared_error(y_train, pred_train, squared=False)
    r2_train = r2_score(y_train, pred_train)

    predicted = cross_val_predict(model, X_Train, y_train, cv=10)
    rmse_cv = mean_squared_error(y_train, predicted, squared=False)
    r2_cv = r2_score(y_train, predicted)

    predicted_test = model.predict(X_test)
    rmse_test = mean_squared_error(y_test, predicted_test, squared=False)
    r2_test = r2_score(y_test, predicted_test)

    return {
        "features": features,
        "model": model,
        "rmse_train": rmse_train,
        "r2_train": r2_train,
        "rmse_cv": rmse_cv,
        "r2_cv": r2_cv,
        "rmse_test": rmse_test,
        "r2_test": r2_test
    }

# Evaluate the model using the specified feature set
print(f"Evaluating feature set: {features}")
try:
    result = evaluate_model(features)
    results = [result]
except ValueError as e:
    if "Input contains NaN" in str(e):
        print(f"Skipping evaluation due to NaN values in feature set: {features}")
    else:
        raise e  # Raise the error if it's not due to NaN values

# Get the trained model
model = results[0]['model']

# Generate predictions for the new dataset
x_pred_st_subset = x_pred_st[features]
predictions = model.predict(x_pred_st_subset)

# Save the predictions to a CSV file
predictions_df = pd.DataFrame(predictions, index=x_pred_st.index, columns=["Predictions"])
predictions_df.to_csv("/content/drive/MyDrive/datasets_Rayeed_GSV_POI_LU/predictions_pm25_svr.csv")

# Display statistics for the predictions
stats = predictions_df.describe()
print(stats)

# Save the statistics to a CSV file
stats.to_csv("/content/drive/MyDrive/datasets_Rayeed_GSV_POI_LU/predictions_pm25_svr_stats.csv")

print(predictions_df.head())
print(stats)
print(results)


[I 2024-08-12 06:59:26,370] A new study created in memory with name: SVR_RMSE
[I 2024-08-12 06:59:26,463] Trial 0 finished with value: 14.198110347165388 and parameters: {'C': 0.02495416503367088, 'epsilon': 3.0762244948072617, 'gamma': 2088.241843908963}. Best is trial 0 with value: 14.198110347165388.
[I 2024-08-12 06:59:26,528] Trial 1 finished with value: 9.735010005394871 and parameters: {'C': 136.06459789071408, 'epsilon': 4.214550259818553, 'gamma': 0.4324606004524325}. Best is trial 1 with value: 9.735010005394871.


Evaluating feature set: ['gsv_tree_p750', 'RdclP_250', 'build_250', 'gsv_sidewalk_p250']


[I 2024-08-12 06:59:26,654] Trial 2 finished with value: 13.909148068211326 and parameters: {'C': 0.44794067985921576, 'epsilon': 8.67521667575906, 'gamma': 0.027200153614933973}. Best is trial 1 with value: 9.735010005394871.
[I 2024-08-12 06:59:27,304] Trial 3 finished with value: 42.616626689217284 and parameters: {'C': 265900.4055211318, 'epsilon': 0.03513974113489729, 'gamma': 0.019283011786673736}. Best is trial 1 with value: 9.735010005394871.
[I 2024-08-12 06:59:27,387] Trial 4 finished with value: 12.954396406367037 and parameters: {'C': 4.604725047316729, 'epsilon': 0.13320154639515158, 'gamma': 0.4352320681893019}. Best is trial 1 with value: 9.735010005394871.
[I 2024-08-12 06:59:27,458] Trial 5 finished with value: 14.111575769784588 and parameters: {'C': 0.15998392754666366, 'epsilon': 1.757229389668096, 'gamma': 0.002920234818650511}. Best is trial 1 with value: 9.735010005394871.
[I 2024-08-12 06:59:27,527] Trial 6 finished with value: 14.178786473012329 and parameters:

       Predictions
count  3724.000000
mean    102.561557
std      11.477525
min      37.862967
25%      95.562826
50%     103.422062
75%     110.738569
max     138.113420
      Predictions
OID_             
1268   111.069034
1271   110.051767
1272   111.727110
1273   110.956306
1274   110.551788
       Predictions
count  3724.000000
mean    102.561557
std      11.477525
min      37.862967
25%      95.562826
50%     103.422062
75%     110.738569
max     138.113420
[{'features': ['gsv_tree_p750', 'RdclP_250', 'build_250', 'gsv_sidewalk_p250'], 'model': SVR(C=42838.994436966896, epsilon=1.688119489123773,
    gamma=0.00013179718337881426), 'rmse_train': 8.116103036862263, 'r2_train': 0.6416743701706706, 'rmse_cv': 8.68528339343536, 'r2_cv': 0.5896534783661267, 'rmse_test': 12.105209589012894, 'r2_test': 0.5302100981089666}]


# **Linear regression**

In [ ]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import cross_val_predict


# Feature set to be used for prediction
features = ['gsv_tree_p750', 'RdclP_250', 'build_250', 'gsv_sidewalk_p250', 'gsv_building_p150']

# Function to run the optimization and evaluate the model
def evaluate_model(features):
    X_Train = X_st[features]
    X_test = X_test_st[features]

    def objective(trial):
        # No hyperparameters to optimize for LinearRegression
        model = LinearRegression()
        predicted = cross_val_predict(model, X_Train, y_train, cv=10)
        return mean_squared_error(y_train, predicted, squared=False)

    study = optuna.create_study(direction='minimize', study_name="LinearRegression_RMSE")
    study.optimize(objective, n_trials=1)  # Only run 1 trial since there are no hyperparameters

    model = LinearRegression()
    model.fit(X_Train, y_train)

    pred_train = model.predict(X_Train)
    rmse_train = mean_squared_error(y_train, pred_train, squared=False)
    r2_train = r2_score(y_train, pred_train)

    predicted = cross_val_predict(model, X_Train, y_train, cv=10)
    rmse_cv = mean_squared_error(y_train, predicted, squared=False)
    r2_cv = r2_score(y_train, predicted)

    predicted_test = model.predict(X_test)
    rmse_test = mean_squared_error(y_test, predicted_test, squared=False)
    r2_test = r2_score(y_test, predicted_test)

    return {
        "features": features,
        "model": model,
        "rmse_train": rmse_train,
        "r2_train": r2_train,
        "rmse_cv": rmse_cv,
        "r2_cv": r2_cv,
        "rmse_test": rmse_test,
        "r2_test": r2_test
    }

# Evaluate the model using the specified feature set
print(f"Evaluating feature set: {features}")
try:
    result = evaluate_model(features)
    results = [result]
except ValueError as e:
    if "Input contains NaN" in str(e):
        print(f"Skipping evaluation due to NaN values in feature set: {features}")
    else:
        raise e  # Raise the error if it's not due to NaN values

# Get the trained model
model = results[0]['model']

# Generate predictions for the new dataset
x_pred_st_subset = x_pred_st[features]
predictions = model.predict(x_pred_st_subset)

# Save the predictions to a CSV file
predictions_df = pd.DataFrame(predictions, index=x_pred_st.index, columns=["Predictions"])
predictions_df.to_csv("/content/drive/MyDrive/datasets_Rayeed_GSV_POI_LU/predictions_pm25_linear_regression.csv")

# Display statistics for the predictions
stats = predictions_df.describe()
print(stats)

# Save the statistics to a CSV file
stats.to_csv("/content/drive/MyDrive/datasets_Rayeed_GSV_POI_LU/predictions_pm25_linear_regression_stats.csv")

print(predictions_df.head())
print(stats)
print(results)


[I 2024-08-20 03:03:03,639] A new study created in memory with name: LinearRegression_RMSE
[I 2024-08-20 03:03:03,738] Trial 0 finished with value: 9.180307460375202 and parameters: {}. Best is trial 0 with value: 9.180307460375202.


Evaluating feature set: ['gsv_tree_p750', 'RdclP_250', 'build_250', 'gsv_sidewalk_p250', 'gsv_building_p150']
       Predictions
count  3724.000000
mean    103.833494
std      12.276296
min      23.669084
25%      96.248882
50%     103.946450
75%     112.023134
max     145.809728
      Predictions
OID_             
1268   104.667336
1271   102.672877
1272   122.027752
1273   102.125521
1274   111.656159
       Predictions
count  3724.000000
mean    103.833494
std      12.276296
min      23.669084
25%      96.248882
50%     103.946450
75%     112.023134
max     145.809728
[{'features': ['gsv_tree_p750', 'RdclP_250', 'build_250', 'gsv_sidewalk_p250', 'gsv_building_p150'], 'model': LinearRegression(), 'rmse_train': 7.591663491572942, 'r2_train': 0.6864861959415453, 'rmse_cv': 9.180307460375202, 'r2_cv': 0.5415444631839184, 'rmse_test': 11.685409714268689, 'r2_test': 0.5622290493767386}]


# **XGBoost**

In [ ]:
import pandas as pd
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import cross_val_predict
from xgboost import XGBRegressor
import optuna

# Feature set to be used for prediction
features = ['gsv_tree_p750', 'RdclP_250', 'build_250', 'gsv_sidewalk_p250']

# Function to run the optimization and evaluate the model
def evaluate_model(features):
    X_Train = X_st[features]
    X_test = X_test_st[features]

    def objective(trial):
        #gamma = trial.suggest_float('gamma', 0, 4000)  # Uncommented to include in optimization
        learning_rate = trial.suggest_float('learning_rate', 0.005, 1.5)
        #max_depth = int(trial.suggest_int('max_depth', 50, 5000))  # Uncommented to include in optimization
        #min_child_weight = int(trial.suggest_int('min_child_weight', 0, 2000))  # Uncommented to include in optimization
        n_estimators = int(trial.suggest_int('n_estimators', 50, 1000))
        reg_alpha = trial.suggest_float('reg_alpha', 0.001, 100)
        reg_lambda = trial.suggest_float('reg_lambda', 0.001, 3)
        #subsample = trial.suggest_float('subsample', 0.6, 1)  # Uncommented to include in optimization

        XGB = XGBRegressor(
            booster='gblinear',
            n_estimators=n_estimators,
            learning_rate=learning_rate,
            reg_alpha=reg_alpha,
            reg_lambda=reg_lambda,
            objective="reg:squarederror",
            seed=42
        )

        predicted = cross_val_predict(XGB, X_Train, y_train, cv=10)
        return mean_squared_error(y_train, predicted, squared=False)

    study = optuna.create_study(direction='minimize', study_name="XGBRegressor_RMSE")
    study.optimize(objective, n_trials=500)  # Run for 500 iterations

    best_params = study.best_params
    model = XGBRegressor(
        booster='gblinear',
        n_estimators=best_params['n_estimators'],
        learning_rate=best_params['learning_rate'],
        reg_alpha=best_params['reg_alpha'],
        reg_lambda=best_params['reg_lambda'],
        objective="reg:squarederror",
        seed=42
    )
    model.fit(X_Train, y_train)

    pred_train = model.predict(X_Train)
    rmse_train = mean_squared_error(y_train, pred_train, squared=False)
    r2_train = r2_score(y_train, pred_train)

    predicted = cross_val_predict(model, X_Train, y_train, cv=10)
    rmse_cv = mean_squared_error(y_train, predicted, squared=False)
    r2_cv = r2_score(y_train, predicted)

    predicted_test = model.predict(X_test)
    rmse_test = mean_squared_error(y_test, predicted_test, squared=False)
    r2_test = r2_score(y_test, predicted_test)

    return {
        "features": features,
        "model": model,
        "rmse_train": rmse_train,
        "r2_train": r2_train,
        "rmse_cv": rmse_cv,
        "r2_cv": r2_cv,
        "rmse_test": rmse_test,
        "r2_test": r2_test
    }

# Evaluate the model using the specified feature set
print(f"Evaluating feature set: {features}")
try:
    result = evaluate_model(features)
    results = [result]
except ValueError as e:
    if "Input contains NaN" in str(e):
        print(f"Skipping evaluation due to NaN values in feature set: {features}")
    else:
        raise e  # Raise the error if it's not due to NaN values

# Get the trained model
model = results[0]['model']

# Generate predictions for the new dataset
x_pred_st_subset = x_pred_st[features]
predictions = model.predict(x_pred_st_subset)

# Save the predictions to a CSV file
predictions_df = pd.DataFrame(predictions, index=x_pred_st.index, columns=["Predictions"])
predictions_df.to_csv("/content/drive/MyDrive/datasets_Rayeed_GSV_POI_LU/predictions_pm25_xgb.csv")

# Display statistics for the predictions
stats = predictions_df.describe()
print(stats)

# Save the statistics to a CSV file
stats.to_csv("/content/drive/MyDrive/datasets_Rayeed_GSV_POI_LU/predictions_pm25_xgb_stats.csv")

print(predictions_df.head())
print(stats)
print(results)


[I 2024-08-20 03:27:37,181] A new study created in memory with name: XGBRegressor_RMSE


Evaluating feature set: ['gsv_tree_p750', 'RdclP_250', 'build_250', 'gsv_sidewalk_p250']


[I 2024-08-20 03:27:37,827] Trial 0 finished with value: 14.238460039802526 and parameters: {'learning_rate': 1.4521684386764837, 'n_estimators': 274, 'reg_alpha': 21.477164539978794, 'reg_lambda': 2.052827112761825}. Best is trial 0 with value: 14.238460039802526.
[I 2024-08-20 03:27:38,519] Trial 1 finished with value: 14.23845927521434 and parameters: {'learning_rate': 0.17776147701431738, 'n_estimators': 931, 'reg_alpha': 16.536380975733625, 'reg_lambda': 2.8054503395948918}. Best is trial 1 with value: 14.23845927521434.
[I 2024-08-20 03:27:38,900] Trial 2 finished with value: 14.238458410879076 and parameters: {'learning_rate': 0.6841625452537871, 'n_estimators': 366, 'reg_alpha': 12.715019164185042, 'reg_lambda': 0.7335343874357471}. Best is trial 2 with value: 14.238458410879076.
[I 2024-08-20 03:27:39,170] Trial 3 finished with value: 14.238458855618372 and parameters: {'learning_rate': 0.23799388026485147, 'n_estimators': 185, 'reg_alpha': 34.62675306459732, 'reg_lambda': 1.2

       Predictions
count  3724.000000
mean    104.356216
std      11.967104
min      37.588955
25%      96.910206
50%     105.148598
75%     113.227707
max     138.906357
      Predictions
OID_             
1268   113.397743
1271   112.742348
1272   115.078247
1273   114.563347
1274   113.477539
       Predictions
count  3724.000000
mean    104.356216
std      11.967104
min      37.588955
25%      96.910206
50%     105.148598
75%     113.227707
max     138.906357
[{'features': ['gsv_tree_p750', 'RdclP_250', 'build_250', 'gsv_sidewalk_p250'], 'model': XGBRegressor(base_score=None, booster='gblinear', callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=1.4270342807571423,
             max_bin=